In [1]:
import os
import io
import pandas as pd
from PIL import Image

In [70]:
import gdown
id = "1hrH0xzsgpmFShRfsqjoS_2EwppN2_u8D"
gdown.download(id=id, output="../resources/data/data.parquet")

Downloading...
From (original): https://drive.google.com/uc?id=1hrH0xzsgpmFShRfsqjoS_2EwppN2_u8D
From (redirected): https://drive.google.com/uc?id=1hrH0xzsgpmFShRfsqjoS_2EwppN2_u8D&confirm=t&uuid=cc8fe048-b737-40ad-98d0-217b6233a04d
To: /home/hieugn/mlops/text-image-retrieval-gke/resources/data/data.parquet
100%|██████████| 563M/563M [00:15<00:00, 36.9MB/s] 


'../resources/data/data.parquet'

In [2]:
data_df = pd.read_parquet(
    "/home/hieugn/mlops/text-image-retrieval-gke/app/resources/data/data.parquet"
)

In [3]:
data_df.head()

,image,artist,genre,style
0,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,16,2,20
1,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,16,4,23
2,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,16,2,20
3,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,16,2,20
4,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,16,2,23


In [4]:
len(data_df)

1131

In [5]:
data_df["index"] = data_df.index

In [7]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [6]:
image_info_df = pd.DataFrame(columns=["image_id", "image_path"])
image_info_df.head()

,image_id,image_path


In [9]:
def extract_data(data_df: pd.DataFrame, image_info_df: pd.DataFrame):
    for i in range(5):
        image_pil = Image.open(io.BytesIO(data_df.iloc[i, 0]["bytes"]))

        width = 500
        ratio = width / float(image_pil.size[0])
        height = int((float(image_pil.size[1]) * float(ratio)))

        path_dir = "../resources/data"
        image_pil.resize((width, height), Image.Resampling.LANCZOS).save(
            f"{path_dir}/images/{data_df.iloc[i, 4]}.jpg"
        )

        image_info_df = pd.concat(
            [
                image_info_df,
                pd.DataFrame(
                    {
                        "image_id": [data_df.iloc[i, 4]],
                        "image_path": [f"{path_dir}/images/{data_df.iloc[i, 4]}.jpg"],
                    }
                ),
            ]
        )

    image_info_df.to_csv(f"{path_dir}/image_info.csv", index=False)

In [ ]:
extract_data(data_df, image_info_df)